In [62]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split

data = pd.read_csv('./data_v3.csv')

def foo(x):
    if x >= 3:
        return 1
    return 0

data['bug_num'] = data['bug_num'].map(lambda x: foo(x) )


In [64]:
'''
created_at,
updated_at,
size,
stargazers_count,
watchers_count, #
forks_count, #
network_count, #
subscribers_count, #
followers, #
public_repos, #
created_at,
contributions, #
author_id,
LOC, #
Add, #
Delete, #
Files, #
bug_num  #
'''

feature_columns = [
    #'watchers_count',
    #'forks_count',
    #'network_count',
    #'subscribers_count',
    'followers',
    'public_repos',
    'contributions',
    'LOC',
    'Add',
    'Delete',
    'Files',
]

target_column = 'bug_num'

train, test = train_test_split(data, test_size=0.25, random_state=14)
xgtrain = xgb.DMatrix(train[feature_columns].values, train[target_column].values)
xgtest = xgb.DMatrix(test[feature_columns].values, test[target_column].values)

print("the train size is {}".format(len(train[target_column].values)))
print("the test size is {}".format(len(test[target_column].values)))

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 2,
    'gamma': 0.1,
    'max_depth': 6,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000,
    'nthread': 4,
}

# build the model
num_round = 300
bst = xgb.train(params, xgtrain, num_round)

# prediction and check the accuracy
preds = bst.predict(xgtest)
labels = xgtest.get_label()

match = 0.0
for i in range(len(preds)):
    if preds[i] == labels[i]:
        match += 1
acc = match / len(preds) % 100.0

print("accuracy rate is {}".format(acc))


the train size is 89
the test size is 30
[19:08:21] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:08:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy rate is 0.9666666666666667
